# Preprocessing

In [1]:
import pandas as pd
text = 'Hi my name is Madi and i like to code!!!'

data = pd.read_csv('./data/train.csv')

In [2]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim import corpora

en_stop = set(stopwords.words('english'))

def get_lemma2(word):
    from nltk.stem.wordnet import WordNetLemmatizer
    """
    This function takes a word and returns its rootword
    """
    return WordNetLemmatizer().lemmatize(word)

def nltk_preprocessing(text):
    """
    This function takes a text and returns a list of tokens
    - lowercase
    - remove short words
    - remove stopwords
    - gets root word (lemma)

    """
    tokens = word_tokenize(text.lower())
    # tokens = [token for token in tokens if len(token) > 4]      # remove short words
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma2(token) for token in tokens]
    return tokens

In [3]:
# Compute bigrams.
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
def make_bigrams(texts):
    bigram = Phrases(texts, min_count=20)
    for idx in range(len(texts)):
        for token in bigram[texts[idx]]:
            if '_' in token:
                # Token is a bigram, add to document.
                texts[idx].append(token)
    return texts



In [4]:
mydict = corpora.Dictionary([nltk_preprocessing(line) for line in data['text']])
corpus = [mydict.doc2bow(nltk_preprocessing(line)) for line in data['text']]

# scipy matrix

In [5]:
# corpus tfidf to scipy sparse matrix
from gensim.matutils import corpus2csc
import scipy.sparse
corpus_sparse = scipy.sparse.csc_matrix(corpus2csc(corpus))

The `corpus sparse matrix` is a scipy sparse matrix. It is a matrix of shape `(n_documents, n_words)` where each cell is the number of times a word appears in a document. The matrix is sparse because most cells are zero.


In [16]:
# convert sparse matrix to coo matrix
from scipy.sparse import coo_matrix
corpus_coo = coo_matrix(corpus_sparse)
print("coo",corpus_coo.shape)

# convert sparse matrix to csr matrix
from scipy.sparse import csr_matrix
corpus_csr = csr_matrix(corpus_sparse)

print("csr",corpus_csr.shape)


coo (21875, 7613)


/home/madis/sandbox/cohere-takehome/venv/lib/python3.8/site-packages/scipy/sparse/compressed.py:80: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arg1 = np.asarray(arg1)


TypeError: no supported conversion for types: (dtype('O'),)

# How to create the TFIDF matrix (corpus) in gensim?
https://www.machinelearningplus.com/nlp/gensim-tutorial/#8howtocreatethetfidfmatrixcorpusingensim

In [7]:
from gensim import models
import numpy as np
# import corpora
from gensim import corpora


# Create the Dictionary and Corpus
mydict = corpora.Dictionary([nltk_preprocessing(line) for line in data['text']])
corpus = [mydict.doc2bow(nltk_preprocessing(line)) for line in data['text']]

# create the tfidf model
tfidf = models.TfidfModel(corpus, smartirs='ntc')


# Implementing LSA using Gensim
https://machinelearninggeek.com/discovering-hidden-themes-of-documents/#:~:text=Implementing%20LSA%20using%20Gensim

In [ ]:
# plot the histogram of tfidf values
import matplotlib.pyplot as plt
tfidf_values = [tfidf[doc] for doc in corpus]
tfidf_values = [item for sublist in tfidf_values for item in sublist]
tfidf_values = [item[1] for item in tfidf_values]

plt.hist(tfidf_values, bins=100)

# add axis 
plt.xlabel('tfidf value')
plt.ylabel('count')

plt.show()
